<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Simple 3-Layered MLP Neural Network</strong></p>

In [7]:
# MNIST handwritten digits classification with 2-layer MLP

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Load the data
data = pd.read_csv("train.csv")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [95]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T # transpose to get the data in a row-wise format
Y_dev = data_dev[0] # first row is the label
X_dev = data_dev[1:n] # rest of the rows are the pixel values
X_dev = X_dev / 255. # normalize pixel values in range 0-1

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape # m_train is the number of training examples and _ is the number of features

In [96]:
Y_train

array([4, 5, 5, ..., 8, 1, 3], dtype=int64)

In [97]:
import numpy as np

def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(10, 10) - 0.5  # Initialize weights for the third layer
    b3 = np.random.rand(10, 1) - 0.5  # Initialize biases for the third layer
    return W1, b1, W2, b2, W3, b3

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y, batch_size):
    one_hot_Y = one_hot(Y, 10)
    dZ3 = A3 - one_hot_Y
    dA2 = W3.T.dot(dZ3)
    dZ2 = dA2 * ReLU_deriv(Z2)
    dW2 = 1 / batch_size * dZ2.dot(A1.T)
    db2 = 1 / batch_size * np.sum(dZ2)
    dA1 = W2.T.dot(dZ2)
    dZ1 = dA1 * ReLU_deriv(Z1)
    dW1 = 1 / batch_size * dZ1.dot(X.T)
    db1 = 1 / batch_size * np.sum(dZ1)
    dW3 = 1 / batch_size * dZ3.dot(A2.T)  # Gradient for the third layer weights
    db3 = 1 / batch_size * np.sum(dZ3)  # Gradient for the third layer biases
    return dW1, db1, dW2, db2, dW3, db3  # Include dW3 and db3 in the return

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3  # Update the third layer weights
    b3 = b3 - alpha * db3  # Update the third layer biases
    return W1, b1, W2, b2, W3, b3  # Include W3 and b3 in the return

In [98]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y, X.shape[1])
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0: 
            print("Iteration: ", i)
            predictions = get_predictions(A3) # A2 because we are using 2 layer MLP
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Gradient Descent</strong></p>

In [99]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 0.1, 501)

Iteration:  0
[5 5 5 ... 7 7 5] [4 5 5 ... 8 1 3]
0.07597560975609756
Iteration:  10
[6 5 5 ... 7 1 5] [4 5 5 ... 8 1 3]
0.14021951219512196
Iteration:  20
[7 5 5 ... 7 1 0] [4 5 5 ... 8 1 3]
0.19053658536585366
Iteration:  30
[7 5 5 ... 3 1 3] [4 5 5 ... 8 1 3]
0.23319512195121953
Iteration:  40
[9 5 5 ... 1 1 3] [4 5 5 ... 8 1 3]
0.2885121951219512
Iteration:  50
[9 5 5 ... 5 1 3] [4 5 5 ... 8 1 3]
0.35617073170731706
Iteration:  60
[4 5 0 ... 0 1 3] [4 5 5 ... 8 1 3]
0.4044146341463415
Iteration:  70
[4 5 0 ... 5 1 3] [4 5 5 ... 8 1 3]
0.44078048780487805
Iteration:  80
[4 0 0 ... 9 1 3] [4 5 5 ... 8 1 3]
0.47168292682926827
Iteration:  90
[4 0 0 ... 8 1 3] [4 5 5 ... 8 1 3]
0.5006585365853659
Iteration:  100
[4 0 0 ... 8 1 3] [4 5 5 ... 8 1 3]
0.525
Iteration:  110
[4 5 0 ... 8 1 3] [4 5 5 ... 8 1 3]
0.5441463414634147
Iteration:  120
[4 5 0 ... 8 1 3] [4 5 5 ... 8 1 3]
0.5619268292682927
Iteration:  130
[4 5 0 ... 8 1 3] [4 5 5 ... 8 1 3]
0.579780487804878
Iteration:  140
[4 5 0 .

In [100]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2, W3, b3)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()
    
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3)
get_accuracy(dev_predictions, Y_dev)

[7 6 2 0 5 1 4 0 0 8 0 8 2 2 6 8 1 1 7 3 8 5 7 0 2 0 1 0 8 7 7 2 3 7 1 4 3
 8 4 8 2 0 3 7 1 9 0 4 9 1 1 8 8 7 1 4 7 6 6 6 9 0 3 6 0 6 7 2 1 1 9 7 1 9
 3 2 4 0 1 6 6 9 1 3 0 1 3 1 6 0 0 0 8 1 6 3 6 2 3 3 8 7 2 9 2 6 8 6 8 9 0
 7 1 7 1 3 7 8 6 1 0 6 1 8 4 1 8 9 3 1 0 9 4 1 1 8 7 5 5 1 1 5 9 0 5 8 2 1
 0 1 9 2 4 0 7 1 4 2 2 6 8 0 6 1 2 0 2 3 0 2 1 2 1 8 9 8 2 1 9 6 8 1 2 8 3
 8 9 6 4 5 6 6 2 0 5 3 0 0 7 5 1 7 6 8 9 1 2 9 7 5 0 5 9 0 7 6 8 4 7 8 7 1
 5 5 3 1 0 3 6 7 1 0 1 1 4 4 8 2 3 6 9 9 0 0 7 8 6 3 2 2 8 7 5 6 3 3 4 7 6
 5 6 8 5 2 6 0 5 1 5 1 0 0 9 7 4 1 9 0 3 7 1 3 1 2 9 2 9 3 2 6 2 0 9 0 6 3
 7 0 8 9 8 2 5 9 9 6 2 3 3 1 1 6 2 8 4 9 2 9 5 4 7 8 4 8 3 1 9 9 5 0 6 3 2
 0 6 7 3 3 8 7 6 2 7 0 4 2 4 9 1 1 4 0 1 6 3 4 0 9 5 2 6 1 4 8 3 8 4 9 6 9
 1 3 8 4 5 6 7 3 2 2 4 2 1 7 8 8 4 2 3 2 6 5 2 9 3 8 7 5 9 8 9 1 8 2 4 2 2
 5 1 6 9 0 0 6 7 2 6 3 0 9 7 5 9 6 9 0 8 3 4 0 0 2 6 2 8 2 5 9 0 2 7 0 8 7
 0 5 7 7 7 4 8 6 5 8 6 0 6 0 8 1 1 0 4 2 9 8 0 5 7 3 0 0 8 0 3 4 6 0 1 0 3
 8 7 0 2 0 6 9 3 2 1 2 2 

0.838

<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Stochastic Gradient Descent</strong></p>

In [111]:
def stochastic_gradient_descent(X, Y, alpha, iterations, batch_size):
    W1, b1, W2, b2, W3, b3 = init_params()
    m = X.shape[1]  # Total number of training examples

    for i in range(iterations):
        # Shuffle the dataset and split it into mini-batches
        permutation = np.random.permutation(m)
        shuffled_X = X[:, permutation]
        shuffled_Y = Y[permutation]

        for j in range(0, m, batch_size):
            # Extract a mini-batch
            X_mini_batch = shuffled_X[:, j:j+batch_size]
            Y_mini_batch = shuffled_Y[j:j+batch_size]

            # Perform forward and backward propagation on the mini-batch
            Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X_mini_batch)
            dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X_mini_batch, Y_mini_batch, batch_size)
            W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)

        if i % 5 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y_mini_batch))
    return W1, b1, W2, b2, W3, b3

In [112]:
W1, b1, W2, b2, W3, b3 = stochastic_gradient_descent(X_train, Y_train, 0.1, 151, 128)

Iteration:  0
[1 4 9 9 3 4 3 0 7 2 1 0 2 9 8 3 9 7 7 0 1 7 0 6 0 0 7 1 9 2 0 5 7 6 2 3 3
 6 1 4] [1 4 9 9 5 4 3 0 7 2 1 0 5 9 8 3 9 5 7 0 1 7 0 6 0 0 7 1 9 6 0 3 7 6 2 7 5
 6 1 6]
0.8
Iteration:  5
[3 4 4 4 5 9 7 2 8 4 0 5 2 4 9 2 0 8 4 2 0 2 0 0 9 6 1 9 0 0 3 9 7 0 3 0 6
 9 4 9] [3 4 4 4 5 9 1 2 8 9 0 5 2 4 4 2 0 1 4 2 0 9 2 0 1 6 1 9 0 0 3 4 7 5 3 0 6
 9 4 9]
0.775
Iteration:  10
[2 4 5 6 8 1 2 4 4 3 2 5 3 5 4 8 9 3 3 4 7 1 7 6 6 6 4 4 0 2 4 4 5 2 8 5 7
 4 2 4] [2 4 5 6 8 1 2 4 4 5 2 5 3 5 4 8 9 3 3 4 7 1 7 6 6 6 9 4 0 2 9 4 5 2 8 5 7
 4 2 4]
0.925
Iteration:  15
[7 7 2 5 0 9 3 6 2 3 3 0 9 6 7 0 1 5 9 2 9 5 8 2 8 6 0 2 0 1 7 9 5 3 3 1 0
 9 7 9] [7 7 2 5 0 9 5 6 2 3 8 0 9 6 7 0 1 5 9 2 9 5 8 2 8 6 0 2 0 1 7 9 5 3 3 1 0
 9 7 9]
0.95
Iteration:  20
[2 2 7 0 1 4 5 4 1 1 7 6 0 4 6 7 2 3 7 7 1 4 3 9 2 8 8 7 0 8 2 0 3 3 6 7 1
 7 9 9] [2 2 7 0 1 4 5 4 1 1 7 6 0 4 6 7 2 3 7 7 1 4 3 9 2 8 8 7 0 8 2 0 3 3 6 7 1
 7 9 9]
1.0
Iteration:  25
[5 8 6 6 2 9 7 8 2 6 8 5 2 9 2 7 6 7 3 3 2 7 0 7 8 9 6 8 

In [113]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3)
get_accuracy(dev_predictions, Y_dev)

[1 6 2 5 5 1 4 0 5 8 0 8 2 2 6 8 2 7 7 3 8 3 7 0 2 0 1 0 2 2 7 2 3 9 1 4 3
 2 4 8 2 0 3 8 1 9 0 9 9 1 1 8 8 7 1 4 7 6 6 6 4 0 3 6 0 6 7 2 1 1 9 7 1 9
 3 2 4 0 8 6 6 8 1 3 0 1 5 1 6 0 0 0 8 1 6 3 6 3 8 8 7 9 8 9 2 6 8 2 8 9 0
 7 1 7 1 3 7 8 6 1 0 6 1 8 4 1 8 9 3 1 0 9 4 1 1 8 7 8 8 1 8 5 9 0 5 8 2 1
 0 1 8 2 4 5 7 1 4 6 2 6 8 0 6 1 2 0 2 3 0 2 1 2 1 5 9 8 2 1 9 6 8 1 2 8 3
 5 9 6 4 0 6 2 2 0 5 3 0 0 7 5 1 7 6 2 9 1 2 9 7 5 0 5 9 0 7 2 8 4 7 8 7 1
 0 7 3 1 0 5 6 7 1 0 1 1 4 4 8 2 3 6 7 9 0 0 7 8 6 3 2 2 8 7 5 4 3 3 4 7 6
 7 6 8 0 3 6 0 5 1 8 1 0 0 9 7 4 1 7 0 3 3 1 3 1 8 9 2 8 3 2 6 2 0 9 0 6 3
 7 0 8 9 8 2 5 9 9 6 8 3 3 1 1 6 2 8 4 9 6 9 5 4 7 8 4 8 3 1 9 9 5 0 6 5 2
 0 8 7 3 3 8 7 6 2 7 0 4 8 4 9 1 1 4 0 1 6 3 9 6 7 2 2 6 1 7 8 5 8 9 9 6 9
 1 3 8 4 8 6 7 3 2 2 2 2 1 7 8 8 4 2 3 2 6 5 2 9 3 8 7 5 9 8 9 1 8 2 4 2 2
 5 1 6 9 0 0 6 7 2 6 3 0 9 7 5 9 6 9 0 8 3 4 0 0 3 6 2 8 2 5 9 0 2 7 5 3 7
 0 5 7 7 7 9 8 6 8 8 6 0 6 0 8 1 1 3 8 2 9 8 0 5 7 3 0 0 8 0 5 4 6 0 1 0 3
 8 9 0 2 0 6 9 3 3 1 2 1 

0.945